# Overview

The agent in this notebook was designed with human readability in mind. All list positions are converted to a custom Vector object for ease of use, so instead of writing `obs['left_team'][obs['active']][0]`, you can simply write: `player.x` to get the controlled player's X position. The main components from the observation are loaded and converted into variables so there isn't much redundant calling of the `obs` value.

# References

These are amazing notebooks which were the core of this agent model. Make sure to check them out.
* https://www.kaggle.com/sx2154/gfootball-rules-from-environment-exploration
* https://www.kaggle.com/eugenkeil/simple-baseline-bot
* https://www.kaggle.com/jaronmichal/human-readable-visualization
* https://www.kaggle.com/david1013/tunable-baseline-bot
* https://www.kaggle.com/yegorbiryukov/gfootball-with-memory-patterns

# Install Environments

In [ ]:
# Install:
# Kaggle environments.
!git clone https://github.com/Kaggle/kaggle-environments.git
!cd kaggle-environments && pip install .

# GFootball environment.
!apt-get update -y
!apt-get install -y libsdl2-gfx-dev libsdl2-ttf-dev

# Make sure that the Branch in git clone and in wget call matches !!
!git clone -b v2.7 https://github.com/google-research/football.git
!mkdir -p football/third_party/gfootball_engine/lib

!wget https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_v2.7.so -O football/third_party/gfootball_engine/lib/prebuilt_gameplayfootball.so
!cd football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install .

# Vector Object

The Vector object used to hold the x, y and z positions of the lists is very useful for readability (Positions with only and X and a Y will be assigned an default Z value of 0 for consistancy). It automatically loads them into the values and has built-in helper functions for useful features.

# Features
* `Vector.x, Vector.y, Vector.z` for values
* `Vector.dist(other: Vector) -> float` -- Find the euclidean distance between two vectors
* `Vector.add(vel: float) -> Vector` -- Adding one vector to another vector
* `Vector.mult(x: float) -> Vector` -- Multplies all values in the object by a scale
* More can be added easily for use later

# Agent

A fairly basic agent model for a core setup of a stronger agent. There is nothing too special in the agent given, but it sets up a base for simple implementation of newer techniques. I found with my experience in the [ConnectX](http://kaggle.com/c/connectx) competition is that it is good to build a strong setup and then fine tune details and features later. This should help with speeding up development in the competition.

In [ ]:
%%writefile agent.py

# Importing Important Imports
from kaggle_environments.envs.football.helpers import *
import math

class Vector:

	'''
	Vector Object
	Parameters: Iterable of length 2 or 3
	'''

	def __init__(self, positions = [0, 0, 0]):
		if len(positions) < 3: positions.append(0)
		self.x, self.y, self.z = positions
		self.vel = None

	def dist(self, other):
		''' Euclidean distance '''
		return math.hypot(other.x - self.x, other.y - self.y)
	
	def add(self, vel):
		''' Adds one vector to the other '''
		return Vector([self.x + vel.x, self.y + vel.y, self.z + vel.z])

	def mult(self, x):
		''' Scales the vector by x '''
		return Vector([self.x * x, self.y * x, self.z * x])

@human_readable_agent
def agent(obs):

	''' Main Agent '''

	# Loading Variables

	N = len(obs['left_team'])

	# Teams
	team = list(map(Vector, obs['left_team']))
	opponents = list(map(Vector, obs['right_team']))

	# Indexes of Active Players
	baller = obs['ball_owned_player']
	active = obs['active']

	# Key Players
	player = team[active]
	goalkeeper = opponents[0]

	# Ball Variables
	ballOwned = (obs['ball_owned_team'] == 0 and active == baller)
	ball = Vector(obs['ball'])
	ball.vel = Vector(obs['ball_direction'])

	# Special Helpers
	sticky = obs['sticky_actions']
	mode = obs['game_mode']

	# Enemy Goal
	target = Vector([1, 0])

	# Directions for movement
	directions = [
		[Action.TopLeft, Action.Top, Action.TopRight],
		[Action.Left, Action.Idle, Action.Right],
		[Action.BottomLeft, Action.Bottom, Action.BottomRight]
	]

	def stickyCheck(action, direction):
		''' Checking for direction and actions '''
		if direction not in sticky:
			return direction
		return action
	
	def dirsign(value):
		''' Getting index for directions '''
		if abs(value) < 0.01: return 1
		elif value < 0: return 0
		return 2

	def getDirection(target, position = player):
		''' Getting direction to move from position to target '''
		xdir = dirsign(target.x - position.x)
		ydir = dirsign(target.y - position.y)
		return directions[ydir][xdir]

	# Always Sprint
	if Action.Sprint not in sticky:
		return Action.Sprint

	# Offense Patterns
	if ballOwned:

		# Special Situations
		if mode in [GameMode.Penalty, GameMode.Corner, GameMode.FreeKick]:
			if player.x > 0: return Action.Shot
			return Action.LongPass

		# Goalkeeper Check
		if baller == 0: 
			return Action.LongPass
		
		# Bad Angle Pass
		if abs(player.y) > 0.2 and player.x > 0.7: 
			return Action.HighPass
			
		# Close to Goalkeeper Shot
		if player.dist(goalkeeper) < 0.4:
			return Action.Shot

		# Goalkeeper is Out
		if goalkeeper.dist(target) > 0.2:
			if player.x > 0:
				return Action.Shot

		#####################
		## Your Ideas Here ##
		#####################

		# Run to Goal
		return getDirection(target)

	# Defensive Patterns
	else:

		# Find the Ball's Next Positions
		nextBall = ball.add(ball.vel.mult(3))

		# Running to the next Ball Position
		if ball.dist(player) > 0.005:
			return getDirection(nextBall)
		
		# Sliding
		elif ball.dist(player) <= 0.005:
			return Action.Slide
		
		# Running Directly at the Ball
		return getDirection(ball)

# Visualizer Code
The code here is from [this beautiful notebook](https://www.kaggle.com/jaronmichal/human-readable-visualization) to show the game. (Collapsed)

In [ ]:
%%writefile visualizer.py
from matplotlib import animation, patches, rcParams
from matplotlib import pyplot as plt
from kaggle_environments.envs.football.helpers import *

WIDTH = 110
HEIGHT = 46.2
PADDING = 10


def initFigure(figwidth=12):
    figheight = figwidth * (HEIGHT + 2 * PADDING) / (WIDTH + 2 * PADDING)

    fig = plt.figure(figsize=(figwidth, figheight))
    ax = plt.axes(xlim=(-PADDING, WIDTH + PADDING), ylim=(-PADDING, HEIGHT + PADDING))
    plt.axis("off")
    return fig, ax


def drawPitch(ax):
    paint = "white"

    # Grass around pitch
    rect = patches.Rectangle((-PADDING / 2, -PADDING / 2), WIDTH + PADDING, HEIGHT + PADDING,
                             lw=1, ec="black", fc="#3f995b", capstyle="round")
    ax.add_patch(rect)

    # Pitch boundaries
    rect = plt.Rectangle((0, 0), WIDTH, HEIGHT, ec=paint, fc="None", lw=2)
    ax.add_patch(rect)

    # Middle line
    plt.plot([WIDTH / 2, WIDTH / 2], [0, HEIGHT], color=paint, lw=2)

    # Dots
    dots_x = [11, WIDTH / 2, WIDTH - 11]
    for x in dots_x:
        plt.plot(x, HEIGHT / 2, "o", color=paint, lw=2)

    # Penalty box
    penalty_box_dim = [16.5, 40.3]
    penalty_box_pos_y = (HEIGHT - penalty_box_dim[1]) / 2

    rect = plt.Rectangle((0, penalty_box_pos_y),
                         penalty_box_dim[0], penalty_box_dim[1], ec=paint, fc="None", lw=2)
    ax.add_patch(rect)
    rect = plt.Rectangle((WIDTH, penalty_box_pos_y), -
                         penalty_box_dim[0], penalty_box_dim[1], ec=paint, fc="None", lw=2)
    ax.add_patch(rect)

    # Goal box
    goal_box_dim = [5.5, penalty_box_dim[1] - 11 * 2]
    goal_box_pos_y = (penalty_box_pos_y + 11)

    rect = plt.Rectangle((0, goal_box_pos_y),
                         goal_box_dim[0], goal_box_dim[1], ec=paint, fc="None", lw=2)
    ax.add_patch(rect)
    rect = plt.Rectangle((WIDTH, goal_box_pos_y),
                         -goal_box_dim[0], goal_box_dim[1], ec=paint, fc="None", lw=2)
    ax.add_patch(rect)

    # Goals
    goal_width = 0.044 / 0.42 * HEIGHT
    goal_pos_y = (HEIGHT / 2 - goal_width / 2)
    rect = plt.Rectangle((0, goal_pos_y), -2, goal_width,
                         ec=paint, fc=paint, lw=2, alpha=0.3)
    ax.add_patch(rect)
    rect = plt.Rectangle((WIDTH, goal_pos_y), 2, goal_width,
                         ec=paint, fc=paint, lw=2, alpha=0.3)
    ax.add_patch(rect)

    # Middle circle
    mid_circle = plt.Circle([WIDTH / 2, HEIGHT / 2], 9.15, color=paint, fc="None", lw=2)
    ax.add_artist(mid_circle)

    # Penalty box arcs
    left = patches.Arc([11, HEIGHT / 2], 2 * 9.15, 2 * 9.15,
                       color=paint, fc="None", lw=2, angle=0, theta1=308, theta2=52)
    ax.add_patch(left)
    right = patches.Arc([WIDTH - 11, HEIGHT / 2], 2 * 9.15, 2 * 9.15,
                        color=paint, fc="None", lw=2, angle=180, theta1=308, theta2=52)
    ax.add_patch(right)

    # Arcs on corners
    corners = [[0, 0], [WIDTH, 0], [WIDTH, HEIGHT], [0, HEIGHT]]
    angle = 0
    for x, y in corners:
        c = patches.Arc([x, y], 2, 2,
                        color=paint, fc="None", lw=2, angle=angle, theta1=0, theta2=90)
        ax.add_patch(c)
        angle += 90


def scale_x(x):
    return (x + 1) * (WIDTH / 2)


def scale_y(y):
    return (y + 0.42) * (HEIGHT / 0.42 / 2)


def extract_data(raw_obs):
    obs = raw_obs[0]["observation"]["players_raw"][0]
    res = dict()
    res["left_team"] = [(scale_x(x), scale_y(y)) for x, y in obs["left_team"]]
    res["right_team"] = [(scale_x(x), scale_y(y)) for x, y in obs["right_team"]]

    ball_x, ball_y, ball_z = obs["ball"]
    res["ball"] = [scale_x(ball_x), scale_y(ball_y), ball_z]
    res["score"] = obs["score"]
    res["steps_left"] = obs["steps_left"]
    res["ball_owned_team"] = obs["ball_owned_team"]

    left_active = raw_obs[0]["observation"]["players_raw"][0]["active"]
    res["left_player"] = res["left_team"][left_active]

    right_active = raw_obs[1]["observation"]["players_raw"][0]["active"]
    res["right_player"] = res["right_team"][right_active]

    res["right_team_roles"] = obs["right_team_roles"]
    res["left_team_roles"] = obs["left_team_roles"]
    res["left_team_direction"] = obs["left_team_direction"]
    res["right_team_direction"] = obs["right_team_direction"]
    res["game_mode"] = GameMode(obs["game_mode"]).name
    return res


def draw_team(obs, team, side):
    x_coords, y_coords = zip(*obs[side])
    team.set_data(x_coords, y_coords)


def draw_ball(obs, ball):
    ball.set_markersize(8 + obs["ball"][2])  # Scale size of ball based on height
    ball.set_data(obs["ball"][:2])


def draw_active_players(obs, left_player, right_player):
    x1, y1 = obs["left_player"]
    left_player.set_data(x1, y1)

    x2, y2 = obs["right_player"]
    right_player.set_data(x2, y2)

    if obs["ball_owned_team"] == 0:
        left_player.set_markerfacecolor("yellow")
        left_player.set_markersize(20)
        right_player.set_markerfacecolor("blue")
        right_player.set_markersize(18)
    elif obs["ball_owned_team"] == 1:
        left_player.set_markerfacecolor("firebrick")
        left_player.set_markersize(18)
        right_player.set_markerfacecolor("yellow")
        right_player.set_markersize(20)
    else:
        left_player.set_markerfacecolor("firebrick")
        left_player.set_markersize(18)
        right_player.set_markerfacecolor("blue")
        right_player.set_markersize(18)


def draw_team_active(obs, team_left_active, team_right_active):
    team_left_active.set_data(WIDTH / 2 - 7, -7)
    team_right_active.set_data(WIDTH / 2 + 7, -7)

    if obs["ball_owned_team"] == 0:
        team_left_active.set_markerfacecolor("indianred")
    else:
        team_left_active.set_markerfacecolor("mistyrose")

    if obs["ball_owned_team"] == 1:
        team_right_active.set_markerfacecolor("royalblue")
    else:
        team_right_active.set_markerfacecolor("lightcyan")


def draw_players_directions(obs, directions, side):
    index = 0
    if "right" in side:
        index = 11
    for i, player_dir in enumerate(obs[f"{side}_direction"]):
        x_dir, y_dir = player_dir
        dist = (x_dir ** 2 + y_dir ** 2)**0.5 + 0.00001  # to prevent division by 0
        x = obs[side][i][0]
        y = obs[side][i][1]
        directions[i + index].set_data([x, x + x_dir / dist], [y, y + y_dir / dist])


def player_actions(step, side):
    if side == 0:
        actions = {0: "idle", 1: "←", 2: "↖", 3: "↑", 4: "↗", 5: "→", 6: "↘", 7: "↓", 8: "↙",
                   9: "l_pass", 10: "h_pass", 11: "s_pass", 12: "shot",
                   13: "sprint", 14: "rel_dir", 15: "rel_spr",
                   16: "slide", 17: "dribble", 18: "stp_drb"}
    else:
        actions = {0: "idle", 1: "→", 2: "↘", 3: "↓", 4: "↙", 5: "←", 6: "↖", 7: "↑", 8: "↗",
                   9: "l_pass", 10: "h_pass", 11: "s_pass", 12: "shot",
                   13: "sprint", 14: "rel_dir", 15: "rel_spr",
                   16: "slide", 17: "dribble", 18: "stp_drb"}

    obs = step[side]["observation"]["players_raw"][0]

    if obs["sticky_actions"][8]:
        spr = "+spr"
    else:
        spr = "-spr"

    if obs["sticky_actions"][9]:
        drb = "+drb"
    else:
        drb = "-drb"

    if 1 in obs["sticky_actions"][0:8]:
        i = obs["sticky_actions"][0:8].index(1) + 1
        drn = actions[i]
    else:
        drn = "|"

    if step[side]["action"]:
        act = actions[step[side]["action"][0]]
    else:
        act = "idle"

    return f"{spr} {drb} {drn} [{act}]".ljust(24, " ")


steps = None
drawings = None
directions = None
ball = left_player = right_player = None
team_left = team_right = None
team_left_active = team_right_active = None
team_left_actions = team_right_actions = None
team_left_number = team_right_number = None
team_left_direction = team_right_direction = None
text_frame = game_mode = match_info = None


def init():
    ball.set_data([], [])
    left_player.set_data([], [])
    right_player.set_data([], [])
    team_left.set_data([], [])
    team_right.set_data([], [])
    team_left_active.set_data([], [])
    team_right_active.set_data([], [])
    return drawings


def animate(i):
    obs = extract_data(steps[i])

    # Draw info about ball possesion
    draw_active_players(obs, left_player, right_player)
    draw_team_active(obs, team_left_active, team_right_active)

    # Draw players
    draw_team(obs, team_left, "left_team")
    draw_team(obs, team_right, "right_team")

    draw_players_directions(obs, directions, "left_team")
    draw_players_directions(obs, directions, "right_team")

    draw_ball(obs, ball)

    # Draw textual informations
    text_frame.set_text(f"Step {i}/{obs['steps_left'] + i - 1}")
    game_mode.set_text(f"{obs['game_mode']} Mode")

    score_a, score_b = obs["score"]
    match_info.set_text(f"{score_a} : {score_b}")

    team_left_actions.set_text(player_actions(steps[i], 0))
    team_right_actions.set_text(player_actions(steps[i], 1))

    team_left_number.set_text(str(steps[i][0]["observation"]["players_raw"][0]["active"]))
    team_right_number.set_text(str(steps[i][1]["observation"]["players_raw"][0]["active"]))

    return drawings


def visualize(trace):
    global steps
    global drawings
    global directions
    global ball, left_player, right_player
    global team_left, team_right
    global team_left_active, team_right_active
    global text_frame, game_mode, match_info
    global team_left_actions, team_right_actions
    global team_left_number, team_right_number
    global team_left_direction, team_right_direction

    rcParams['font.family'] = 'monospace'
    rcParams['font.size'] = 12

    steps = trace

    fig, ax = initFigure()
    drawPitch(ax)
    ax.invert_yaxis()

    left_player, = ax.plot([], [], "o", ms=18, mfc="firebrick", mew=0, alpha=0.5)
    right_player, = ax.plot([], [], "o", ms=18, mfc="blue", mew=0, alpha=0.5)
    team_left, = ax.plot([], [], "o", ms=12, mfc="firebrick", mew=1, mec="white")
    team_right, = ax.plot([], [], "o", ms=12, mfc="blue", mew=1, mec="white")
    ball, = ax.plot([], [], "o", ms=8, mfc="wheat", mew=1, mec="black")

    team_left_active, = ax.plot([], [], "o", ms=16, mfc="mistyrose", mec="None")
    team_right_active, = ax.plot([], [], "o", ms=16, mfc="lightcyan", mec="None")

    textheight = -6
    text_frame = ax.text(-5, textheight, "", ha="left")
    match_info = ax.text(WIDTH / 2, textheight, "", ha="center", fontweight="bold")
    game_mode = ax.text(WIDTH + 5, textheight, "", ha="right")

    team_left_actions = ax.text(WIDTH / 4 + 2, textheight, "", ha="center")
    team_right_actions = ax.text(3 * WIDTH / 4 + 2, textheight, "", ha="center")

    team_left_number = ax.text(WIDTH / 2 - 7, -6.3, "", ha="center", fontsize=10)
    team_right_number = ax.text(WIDTH / 2 + 7, -6.3, "", ha="center", fontsize=10)

    # Drawing of directions definitely can be done in a better way
    directions = []
    for _ in range(22):
        direction, = ax.plot([], [], color="yellow", lw=1.5)
        directions.append(direction)

    drawings = [team_left_active, team_right_active, left_player, right_player,
                team_left, team_right, ball, text_frame, match_info,
                game_mode, team_left_actions, team_right_actions, team_left_number, team_right_number]

    fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=None, hspace=None)
    anim = animation.FuncAnimation(fig, animate, init_func=init, blit=True,
                                   interval=100, frames=len(steps), repeat=True)
    return anim

# Running the Agent

In [ ]:
from kaggle_environments import make
from kaggle_environments import agent

from visualizer import visualize
from IPython.display import HTML

env = make("football", configuration={"save_video": True, "scenario_name": "11_vs_11_kaggle", "running_in_notebook": True}, debug = True)
output = env.run(['agent.py', 'do_nothing'])
scores = output[-1][0]["observation"]["players_raw"][0]["score"]

print("Scores  {0} : {1}".format(*scores))

viz = visualize(output)
HTML(viz.to_html5_video())

# Conclusion
I'd love to hear your guys' thoughts about this idea. Feel free to ask questions and share your score! Feedback and ideas are always welcome, and thank you Kaggle for this amazing opportunity!